# Assignment for #05, 06, and 07

Downloading auxiliary classes:

In [ ]:
%%bash
wget --no-check-certificate -O "chap3lib.7z" "https://docs.google.com/uc?export=download&id=1SuDvAXGqR17finK7rO9xsqQh2Hey1VVQ"
7z x chap3lib.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 3919 bytes (4 KiB)

Extracting archive: chap3lib.7z
--
Path = chap3lib.7z
Type = 7z
Physical Size = 3919
Headers Size = 130
Method = LZMA2:24k
Solid = -
Blocks = 1

Everything is Ok

Size:       19281
Compressed: 3919


--2025-05-20 14:07:13--  https://docs.google.com/uc?export=download&id=1SuDvAXGqR17finK7rO9xsqQh2Hey1VVQ
Resolving docs.google.com (docs.google.com)... 74.125.132.102, 74.125.132.101, 74.125.132.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.132.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1SuDvAXGqR17finK7rO9xsqQh2Hey1VVQ&export=download [following]
--2025-05-20 14:07:13--  https://drive.usercontent.google.com/download?id=1SuDvAXGqR17finK7rO9xsqQh2Hey1VVQ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.201.132, 2607:f8b0:4001:c01::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.201.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3919 (3.8K) [application/octet-stream]
Saving to: ‘chap3lib.7z’

     0K ...                                                   100% 27.7M=0s

Importing auxiliary classes:

In [ ]:
from chap3lib import *

## A1.

**Objective:**

The goal of this assignment is to define the `Exp2Alg` interface, which defines the operations for constructing expressions in the `Exp2` language. You will need to fill in the methods that currently raise `NotImplementedError()`.

**Background:**

The `Exp2` language is a small functional kernel whose syntax includes constants (integers and booleans), variables, unary and binary operations, conditional expressions, local bindings, function definitions (including recursive ones), and function applications. Each of these syntactic constructs corresponds to a method in the `Exp2Alg` interface.

Object Algebras provide a way to define language syntax and semantics separately. The `Exp2Alg` acts as an interface (algebra) that defines operations for each language construct of `Exp2`.

In [ ]:
class Exp2Alg:
  def cint(self, v):          raise NotImplementedError()
  def cbool(self, v):         raise NotImplementedError()
  def var(self, x):           raise NotImplementedError()
  def _not(self, e):          raise NotImplementedError()
  def plus(self, e1, e2):     raise NotImplementedError()
  def minus(self, e1, e2):    raise NotImplementedError()
  def times(self, e1, e2):    raise NotImplementedError()
  def div(self, e1, e2):      raise NotImplementedError()
  def mult(self, e1, e2):     raise NotImplementedError()
  def _and(self, e1, e2):     raise NotImplementedError()
  def _or(self, e1, e2):      raise NotImplementedError()
  def eq(self, e1, e2):       raise NotImplementedError()
  def le(self, e1, e2):       raise NotImplementedError()
  def cond(self, e1, e2, e3): raise NotImplementedError()
  def let(self, x, e1, e2):   raise NotImplementedError()
  def fun(self, x, e):        raise NotImplementedError()
  def rec(self, f, x, e):     raise NotImplementedError()
  def app(self, e1, e2):      raise NotImplementedError()

## A2.

**Objective:**

The goal of this assignment is to practice representing `Exp2` language constructs using the Object Algebra pattern. You will define a Python function that takes an instance of an `Exp2Alg` implementation and uses its methods to construct the `Exp2` expression `let x = 10 in (x + 5)`.

**Background:**

As discussed in the lecture, `Exp2` expressions can be programmatically constructed in Python by calling methods on an object that implements the `Exp2Alg` interface. Each method on this `alg` object corresponds to a specific syntactic construct in the `Exp2` language.

For example:
* A constant integer like `10` would be created using `alg.cint(10)`.
* A variable like `x` would be created using `alg.var("x")`.
* An addition expression like `e1 + e2` would be created using `alg.plus(e1, e2)`.
* A let-binding expression `let x = e1 in e2` would be created using `alg.let("x", e1, e2)`.

**Task:**

Define a Python function, let's call it `let_example`, that takes a single argument:
* `alg`: An instance of a class that implements the `Exp2Alg` interface.

This function should use the methods of the `alg` object to construct and return the representation of the following `Exp2` expression:

`let x = 10 in (x + 5)`

In [ ]:
def let_example(alg):
    return alg.let("x", alg.cint(10), alg.plus(alg.var("x"), alg.cint(5)))

## A3.

**Objective:**

The primary goal of this assignment is to implement a call-by-value evaluation function for the `Exp2` language. You will achieve this by creating a Python class named `Exp2Eval` that implements the `Exp2Alg` interface. The methods in your `Exp2Eval` class will not directly return values, but rather "computations" – functions that, when given an environment, will produce the evaluated result (a value or an error).

**Background:**

The call-by-value evaluation rules for `Exp2` expressions are specified formally as $[[e]]_{Env}$ on the lecture slides. A strategy for implementing such an evaluator using object algebras, where algebra methods return computations (functions), is also discussed on the slides. This is the strategy you must follow.

You will use parts of the provided `chap3lib.py` for this assignment, including:
* Value classes: `Value`, `VInt2`, `VBool2`, `Clos2`, and `ClosRec2`.
* Error classes: `Err2`, `E_Typ2` (type error), `E_Exec2` (execution error, e.g., division by zero), `E_Undef2` (unbound variable).
* Environment lookup function: `lookup_env(env, name)`.

In [ ]:
class Eval2Alg(Exp2Alg):
    def cint(self, v):
        def _eval(env):
            return VInt2(v)
        return _eval

    def cbool(self, v):
        def _eval(env):
            return VBool2(v)
        return _eval

    def var(self, x):
        def _eval(env):
            val = lookup_env(env, x)
            if val is None: raise E_Undef2()
            return val
        return _eval

    def _not(self, e):
        def _eval(env):
            v = e(env)
            if not isinstance(v, VBool2): raise E_Typ2()
            return VBool2(not v.v)
        return _eval

    def plus(self, e1, e2):
        def _eval(env):
            v1 = e1(env)
            v2 = e2(env)
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2): raise E_Typ2()
            return VInt2(v1.v + v2.v)
        return _eval

    def minus(self, e1, e2):
        def _eval(env):
            v1 = e1(env)
            v2 = e2(env)
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2): raise E_Typ2()
            return VInt2(v1.v - v2.v)
        return _eval

    def let(self, x, e1, e2):
        def _eval(env):
            v1 = e1(env)
            new_env = [(x, v1)] + env
            return e2(new_env)
        return _eval

    def fun(self, x, e):
        def _eval(env):
            return Clos2(x, e, env)
        return _eval

    def rec(self, f, x, e):
        def _eval(env):
            return ClosRec2(f, x, e, env)
        return _eval

    def app(self, e1, e2):
        def _eval(env):
            v_fun = e1(env)
            v_arg = e2(env)

            if isinstance(v_fun, Clos2):
                app_env = [(v_fun.x, v_arg)] + v_fun.env
                return v_fun.e(app_env)
            elif isinstance(v_fun, ClosRec2):
                app_env = [(v_fun.f, v_fun), (v_fun.x, v_arg)] + v_fun.env
                return v_fun.e(app_env)
            else:
                raise E_Typ2()
        return _eval

    def le(self, e1, e2):
        def _eval(env):
            v1 = e1(env)
            v2 = e2(env)
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2):
                raise E_Typ2()
            return VBool2(v1.v <= v2.v)
        return _eval

    def div(self, e1, e2):
        def _eval(env):
            v1 = e1(env)
            v2 = e2(env)
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2):
                raise E_Typ2()
            if v2.v == 0:
                raise E_Exec2()
            return VInt2(v1.v // v2.v)
        return _eval

    def cond(self, e1, e2, e3):
        def _eval(env):
            cond_val = e1(env)
            if not isinstance(cond_val, VBool2):
                raise E_Typ2()
            if cond_val.v:
                return e2(env)
            else:
                return e3(env)
        return _eval

    def mult(self, e1, e2):
        def _eval(env):
            v1 = e1(env)
            v2 = e2(env)
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2):
                raise E_Typ2()
            return VInt2(v1.v * v2.v)
        return _eval

## A4.

**Objective:**

The goal of this assignment is to implement a derivation tree generator for the `Exp2` language under Call-by-Value operational semantics. You will create a Python class `Exp2EvalTreeGenerator` that implements the `Exp2Alg` interface. Instead of directly evaluating to a final value, each method in this algebra will contribute to constructing a representation of the expression's Abstract Syntax Tree (AST) and a "computation" that, when executed with an environment, produces an `EvalStepNode`. This `EvalStepNode` (or one of its specific subclasses `F1` through `F20` from `chap3lib.py`) represents the root of the derivation (sub)tree for that part of the expression.

**Background:**

* The `Exp2` language syntax and its `Exp2Alg` interface are defined in the lecture slides.
* The Call-by-Value operational semantics rules (F1-F20) are described in the lecture slides.
* The provided `chap3lib.py` file contains:
    * AST node classes (`ANode`, `AConst`, `AVar`, `AUnOpExp`, `ABinOpExp`, `ACond`, `ALet`, `AFun`, `ARec`, `AApp`, and specific operator classes like `APlus`, `ANotOp`).
    * Value classes (`Val2`, `VInt2`, `VBool2`) and Error classes (`Err2`, `E_Typ2`, `E_Exec2`, `E_Undef2`).
    * Environment utilities (`lookup_env`).
    * Closure classes `Clos2` and `ClosRec2` which store an `ANode` for the body and a callable to get an `EvalStepNode`. These are suitable for this assignment.
    * `EvalStepNode` class and its subclasses `F1` through `F20`, which you **must use** to represent each step in the derivation tree. Each `Fk` class constructor typically requires the current environment, the AST node of the expression being evaluated (`expr_ast`), the resulting value/error, and any premise `EvalStepNode`s.

In [11]:
class Exp2EvalTreeGenerator(Exp2Alg):
    def __init__(self):
        pass

    def cint(self, value):
        return (
            f"cint({value})",
            lambda env: EvalStepNode(
                rule_name=f"cint({value})",
                env=env,
                expr_ast=f"cint({value})",
                result_value=VInt2(value),
                premises=[]
            )
        )

    def var(self, name):
        return (
            f"var({name})",
            lambda env: EvalStepNode(
                rule_name=f"var({name})",
                env=env,
                expr_ast=f"var({name})",
                result_value=lookup_env(env, name),
                premises=[]
            )
        )

    def plus(self, e1, e2):
        s1, f1 = e1
        s2, f2 = e2
        def eval_plus(env):
            node1 = f1(env)
            node2 = f2(env)
            v1, v2 = node1.result_value, node2.result_value
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2):
                raise TypeError("Expected VInt2")
            return EvalStepNode(
                rule_name="plus",
                env=env,
                expr_ast=f"({s1} + {s2})",
                result_value=VInt2(v1.v + v2.v),
                premises=[node1, node2]
            )
        return (f"({s1} + {s2})", eval_plus)

    def minus(self, e1, e2):
        s1, f1 = e1
        s2, f2 = e2
        def eval_minus(env):
            node1 = f1(env)
            node2 = f2(env)
            v1, v2 = node1.result_value, node2.result_value
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2):
                raise TypeError("Expected VInt2")
            return EvalStepNode(
                rule_name="minus",
                env=env,
                expr_ast=f"({s1} - {s2})",
                result_value=VInt2(v1.v - v2.v),
                premises=[node1, node2]
            )
        return (f"({s1} - {s2})", eval_minus)

    def mul(self, e1, e2):
        s1, f1 = e1
        s2, f2 = e2
        def eval_mul(env):
            node1 = f1(env)
            node2 = f2(env)
            v1, v2 = node1.result_value, node2.result_value
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2):
                raise TypeError("Expected VInt2")
            return EvalStepNode(
                rule_name="mul",
                env=env,
                expr_ast=f"({s1} * {s2})",
                result_value=VInt2(v1.v * v2.v),
                premises=[node1, node2]
            )
        return (f"({s1} * {s2})", eval_mul)

    def div(self, e1, e2):
        s1, f1 = e1
        s2, f2 = e2
        def eval_div(env):
            node1 = f1(env)
            node2 = f2(env)
            v1, v2 = node1.result_value, node2.result_value
            if not isinstance(v1, VInt2) or not isinstance(v2, VInt2):
                raise TypeError("Expected VInt2")
            if v2.v == 0:
                return EvalStepNode(
                    rule_name="div_zero",
                    env=env,
                    expr_ast=f"({s1} / {s2})",
                    result_value=E_Exec2(),
                    premises=[node1, node2]
                )
            return EvalStepNode(
                rule_name="div",
                env=env,
                expr_ast=f"({s1} / {s2})",
                result_value=VInt2(v1.v // v2.v),
                premises=[node1, node2]
            )
        return (f"({s1} / {s2})", eval_div)

    def le(self, e1, e2):
        s1, f1 = e1
        s2, f2 = e2
        def eval_le(env):
            node1 = f1(env)
            node2 = f2(env)
            v1, v2 = node1.result_value, node2.result_value
            return EvalStepNode(
                rule_name="le",
                env=env,
                expr_ast=f"({s1} <= {s2})",
                result_value=int(v1.v <= v2.v),
                premises=[node1, node2]
            )
        return (f"({s1} <= {s2})", eval_le)

    def fun(self, x, e):
        s, f = e
        def eval_fun(env):
            return EvalStepNode(
                rule_name=f"fun({x})",
                env=env,
                expr_ast=f"fun({x}, {s})",
                result_value=(x, e),
                premises=[]
            )
        return (f"fun({x}, {s})", eval_fun)

    def app(self, e1, e2):
        s1, f1 = e1
        s2, f2 = e2
        def eval_app(env):
            node1 = f1(env)
            node2 = f2(env)
            fun_val = node1.result_value
            arg_node = node2
            result = self.apply_fun(fun_val, arg_node, env)
            return EvalStepNode(
                rule_name="app",
                env=env,
                expr_ast=f"app({s1}, {s2})",
                result_value=result,
                premises=[node1, node2]
            )
        return (f"app({s1}, {s2})", eval_app)

    def apply_fun(self, fun_val, arg_node, env):
        if callable(fun_val):
            return fun_val(env, arg_node.result_value)
        if isinstance(fun_val, tuple) and len(fun_val) == 2:
            x, e = fun_val
            new_env = [(x, arg_node.result_value)] + env
            return e[1](new_env).result_value
        raise ValueError("Invalid function application")

    def let(self, x, e1, e2):
        s1, f1 = e1
        s2, f2 = e2
        def eval_let(env):
            node1 = f1(env)
            env1 = [(x, node1.result_value)] + env
            node2 = f2(env1)
            return EvalStepNode(
                rule_name="let",
                env=env,
                expr_ast=f"let({x} = {s1} in {s2})",
                result_value=node2.result_value,
                premises=[node1, node2]
            )
        return (f"let({x} = {s1} in {s2})", eval_let)

    def mult(self, e1, e2):
        return self.mul(e1, e2)

    def cond(self, e1, e2, e3):
        s1, f1 = e1
        s2, f2 = e2
        s3, f3 = e3
        def eval_cond(env):
            cond_node = f1(env)
            if cond_node.result_value:
                then_node = f2(env)
                return EvalStepNode(
                    rule_name="cond-true",
                    env=env,
                    expr_ast=f"cond({s1}, {s2}, {s3})",
                    result_value=then_node.result_value,
                    premises=[cond_node, then_node]
                )
            else:
                else_node = f3(env)
                return EvalStepNode(
                    rule_name="cond-false",
                    env=env,
                    expr_ast=f"cond({s1}, {s2}, {s3})",
                    result_value=else_node.result_value,
                    premises=[cond_node, else_node]
                )
        return (f"cond({s1}, {s2}, {s3})", eval_cond)

    def rec(self, f, x, e):
        s, feval = e
        def eval_rec(env):
            def func(env_call, arg_value):
                new_env = [(f, (x, e))] + [(x, arg_value)] + env
                return feval(new_env).result_value
            return EvalStepNode(
                rule_name=f"rec {f}",
                env=env,
                expr_ast=f"rec {f}({x}) = {s}",
                result_value=func,
                premises=[]
            )
        return (f"rec {f}({x}) = {s}", eval_rec)

You can check your `Exp2EvalTreeGenerator` is correct or not by:
```python
tree = let_example(Exp2EvalTreeGenerator())
print(tree.pretty_print())
```
The above code will generate LaTeX code to render the derivation tree.

## A5.

**Objective:**

The goal of this assignment is to define and implement a Call-by-Name (CbN) evaluation function for the `Exp2` language. This evaluation function must be consistent with the Call-by-Name operational semantics discussed in the lecture slides (rules F'11-F'17). You will implement this evaluator as a Python class, say `CbNEval2Alg`, that is a subclass of `Exp2Alg`.

**Background:**

Call-by-Name evaluation differs significantly from Call-by-Value. Key principles include:
* **Delayed Evaluation:** Arguments to functions and expressions bound in `let` are not evaluated until their value is actually needed.
* **Thunks:** Instead of passing values, CbN passes "thunks" (or "frozen expressions"). A thunk encapsulates an unevaluated expression and the environment in which it was defined. You should define `Thunk` class to represent them as a subclass of `Value`.
* **Environment:** The evaluation environment will bind variables not only to direct values (like integers or booleans if they've been forced) but also to these thunks.

You should refer to the lecture slides:
* Call-by-Name operational semantics (focusing on rules F'11, F'13, F'14, F'15, F'16).
* The general strategy of algebra methods returning computations (functions from environment to result) is still applicable, but the nature of these computations and the values they handle will change for CbN.

You will use parts of the provided `chap3lib.py`:
* Value classes: `Val2` (as a base), `VInt2`, `VBool2`.
* Error classes: `Err2`, `E_Typ2`, `E_Exec2`, `E_Undef2`.

In [13]:
class Thunk(Val2):
    def __init__(self, expr_ast, env, eval_func):
        self.expr_ast = expr_ast
        self.env = env
        self.eval_func = eval_func
        self._cached = None

    def force(self):
        if self._cached is None:
            self._cached = self.eval_func(self.env)
        return self._cached

class CbNEval2Alg(Exp2Alg):
    def cint(self, value):
        ast = AConst(value)
        def eval_cint(env):
            return EvalStepNode("F'11", env, ast, VInt2(value), [])
        return (ast, eval_cint)

    def cbool(self, value):
        ast = AConst(value)
        def eval_cbool(env):
            return EvalStepNode("F'12", env, ast, VBool2(value), [])
        return (ast, eval_cbool)

    def var(self, name):
        ast = AVar(name)
        def eval_var(env):
            val = lookup_env(env, name)
            if isinstance(val, Thunk):
                val = val.force()
            return EvalStepNode("F'13", env, ast, val.result_value, [val])
        return (ast, eval_var)

    def add(self, e1, e2):
        e1_ast, e1_eval = e1
        e2_ast, e2_eval = e2
        ast = AAdd(e1_ast, e2_ast)

        def eval_add(env):
            v1_node = e1_eval(env)
            v1 = v1_node.result_value
            if not isinstance(v1, VInt2):
                raise E_Typ2("Expected integer")

            v2_node = e2_eval(env)
            v2 = v2_node.result_value
            if not isinstance(v2, VInt2):
                raise E_Typ2("Expected integer")

            result = VInt2(v1.value + v2.value)
            return EvalStepNode("F'17", env, ast, result, [v1_node, v2_node])

        return (ast, eval_add)

    def let(self, name, e1, e2):
        e1_ast, e1_eval = e1
        e2_ast, e2_eval = e2
        ast = ALet(name, e1_ast, e2_ast)

        def eval_let(env):
            thunk = Thunk(e1_ast, env, e1_eval)
            new_env = extend_env(env, name, thunk)
            e2_node = e2_eval(new_env)
            return EvalStepNode("F'14", env, ast, e2_node.result_value, [e2_node], bound_name=name, bound_expr=e1_ast)

        return (ast, eval_let)

    def iff(self, e1, e2, e3):
        e1_ast, e1_eval = e1
        e2_ast, e2_eval = e2
        e3_ast, e3_eval = e3
        ast = AIf(e1_ast, e2_ast, e3_ast)

        def eval_if(env):
            cond_node = e1_eval(env)
            cond_val = cond_node.result_value
            if not isinstance(cond_val, VBool2):
                raise E_Typ2("Expected boolean")

            if cond_val.value:
                then_node = e2_eval(env)
                return EvalStepNode("F'16-true", env, ast, then_node.result_value, [cond_node, then_node])
            else:
                else_node = e3_eval(env)
                return EvalStepNode("F'16-false", env, ast, else_node.result_value, [cond_node, else_node])

        return (ast, eval_if)

    def fun(self, param, body):
        ast = AFun(param, body)
        def eval_fun(env):
            clos = Clos2(param, body, env)
            return EvalStepNode("F'15", env, ast, clos, [])
        return (ast, eval_fun)

    def app(self, e1, e2):
        e1_ast, e1_eval = e1
        e2_ast, e2_eval = e2
        ast = AApp(e1_ast, e2_ast)

        def eval_app(env):
            fun_node = e1_eval(env)
            fun_val = fun_node.result_value
            if not isinstance(fun_val, Clos2):
                raise E_Typ2("Expected function")

            thunk = Thunk(e2_ast, env, e2_eval)
            new_env = extend_env(fun_val.env, fun_val.param, thunk)

            body_alg = CbNEval2Alg()
            body_ast, body_eval = fun_val.body.accept(body_alg)
            body_node = body_eval(new_env)
            return EvalStepNode("F'16", env, ast, body_node.result_value, [fun_node, body_node])

        return (ast, eval_app)

    def rec(self, name, expr):
        expr_ast, expr_eval = expr
        ast = ARec(name, expr_ast)

        def eval_rec(env):
            thunk = Thunk(ast, env, expr_eval)
            new_env = extend_env(env, name, thunk)
            val_node = expr_eval(new_env)
            return EvalStepNode("F'17-rec", env, ast, val_node.result_value, [val_node])

        return (ast, eval_rec)